In [ ]:
import numpy as np
import scipy
import torch

u = torch.scalar_tensor(0., requires_grad=True)
v = torch.scalar_tensor(0., requires_grad=True)
sigma = 0.01


def error(u, v):
    return torch.exp(u) + torch.exp(2 * v) + torch.exp(u * v) + u ** 2 - 2 * u * v + 2 * v ** 2 - 3 * u - 2 * v


for x in range(5):
    E = error(u, v)
    E.backward()
    print("grad u v:", u.grad, v.grad)
    u.data = u.data - sigma * u.grad
    v.data = v.data - sigma * v.grad
    u.grad.zero_()
    v.grad.zero_()
    print("data u v:", u, v)
print(error(u, v).item())

In [ ]:
u = torch.scalar_tensor(0., requires_grad=True)
v = torch.scalar_tensor(0., requires_grad=True)
Y = error(u, v)

# Y.backward(retain_graph=True)
Y.backward(retain_graph=True)
print(u.grad, v.grad)


In [ ]:
import numpy as np


def grad_u(u, v):
    return np.exp(u) + np.exp(u * v) * v + 2 * u - 2 * v - 3


def grad_v(u, v):
    return np.exp(2 * v) * 2 + np.exp(u * v) * u - 2 * u + 4 * v - 2


def cal(u, v):
    return np.exp(u) + np.exp(2 * v) + np.exp(u * v) + u ** 2 - 2 * u * v + 2 * v ** 2 - 3 * u - 2 * v


u = 0
v = 0
sigma = 0.01
for _ in range(5):
    u -= sigma * grad_u(u, v)
    v -= sigma * grad_v(u, v)
    print(cal(u, v))

In [ ]:
import numpy as np

f = lambda X: np.sign(X[:, 0] ** 2 + X[:, 1] ** 2 - 0.6)


def experience():
    size = 1000
    r = np.random.rand(size, 2) * 2 + -1
    y = f(r)
    y[: int(size * 0.1)] *= -1
    data = np.hstack((np.ones([size, 1]), r, y.reshape(-1, 1)))
    np.random.shuffle(data)
    X = data[:, :-1]
    y = data[:, -1]
    y = y.reshape((-1, 1))
    w = np.dot(np.linalg.pinv(X), y).reshape((-1, 1))
    y_hat = np.sign(np.dot(X, w))
    e_in = np.sum(y_hat != y) / y.size
    return e_in


e_sum = 0
for _ in range(1000):
    e_sum += experience()
e_ava = e_sum / 1000
print(e_ava)


In [ ]:
import numpy as np

f = lambda X: np.sign(X[:, 0] ** 2 + X[:, 1] ** 2 - 0.6)


def experience():
    size = 1000
    r = np.random.rand(size, 2) * 2 + -1
    y = f(r)
    y[: int(size * 0.1)] *= -1
    data = np.hstack((np.ones([size, 1]), r, y.reshape(-1, 1)))
    np.random.shuffle(data)
    X = data[:, :-1]
    x1 = X[:, 1].reshape(-1, 1)
    x2 = X[:, 2].reshape(-1, 1)
    X = np.hstack((X, x1 * x2, x1 ** 2, x2 ** 2))
    y = data[:, -1]
    y = y.reshape((-1, 1))
    w = np.dot(np.linalg.pinv(X), y).reshape((-1, 1))
    print(w)
    y_hat = np.sign(np.dot(X, w))
    e_in = np.sum(y_hat != y) / y.size
    return e_in


experience()

In [ ]:
import numpy as np

f = lambda X: np.sign(X[:, 0] ** 2 + X[:, 1] ** 2 - 0.6)


def generate():
    size = 1000
    r = np.random.rand(size, 2) * 2 + -1
    y = f(r)
    y[: int(size * 0.1)] *= -1
    data = np.hstack((np.ones([size, 1]), r, y.reshape(-1, 1)))
    np.random.shuffle(data)
    X = data[:, :-1]
    x1 = X[:, 1].reshape(-1, 1)
    x2 = X[:, 2].reshape(-1, 1)
    X = np.hstack((X, x1 * x2, x1 ** 2, x2 ** 2))
    y = data[:, -1]
    y = y.reshape((-1, 1))
    return X, y


def experience():
    X, y = generate()
    w = np.dot(np.linalg.pinv(X), y).reshape((-1, 1))
    y_hat = np.sign(np.dot(X, w))
    e_in = np.sum(y_hat != y) / y.size
    X_out, y_out = generate()
    y_out_hat = np.sign(np.dot(X_out, w))
    e_out = np.sum(y_out_hat != y_out) / y_out.size
    return e_out


e_out_sum = 0
for _ in range(1000):
    e_out_sum += experience()
print(e_out_sum / 1000)

In [123]:
import os
import wget
from tqdm import tqdm
from scipy.special import expit

if not os.path.exists('hw3_train.dat'):
    train_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw3_train.dat"
    test_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw3_test.dat"
    wget.download(train_url)
    wget.download(test_url)
train_data = np.loadtxt('hw3_train.dat')
test_data = np.loadtxt('hw3_test.dat')

X, y = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]
X = np.hstack((np.ones([X.shape[0],1]),X))
X_test = np.hstack((np.ones([X_test.shape[0],1]),X_test))
w = np.zeros([X.shape[1],1])
rate = 0.01
times = 2000
for _ in tqdm(range(times)):
    grads = np.dot(1/(np.exp(y.reshape(1,-1) * np.dot(w.T,X.T).reshape(1,-1)) + 1 ), - y.reshape((-1,1)) * X).reshape(-1,1)/y.size
    # grads = np.dot(expit(-y.reshape(1,-1) * np.dot(w.T,X.T).reshape(1,-1)) , - y.reshape((-1,1)) * X).reshape(-1,1)/y.size
    w -= rate * grads
y_predict = np.dot(X_test, w).squeeze()
print(y_predict)
print(expit(y_predict))

y_predict = np.where(expit(y_predict) > 0.5, 1.0, -1.0)

e_out = np.sum(y_predict != y_test)/y_test.size
print(e_out)





  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:00<00:00, 10425.51it/s]

[ 0.51055671 -0.04094096  0.3639267  ...  0.80367157  0.0821796
  0.27358699]
[0.62493697 0.48976619 0.58999064 ... 0.69075932 0.52053334 0.56797329]
0.22


In [ ]:
y = np.array([1,2,3,4]).reshape(1,-1)
x = np.array([1,2,3,4]).reshape(1,-1)
c = -y * x
print(c)

In [127]:
print(np.random.rand())



0.800056217469395


array([[1., 1., 1.],
       [1., 4., 1.],
       [1., 1., 1.]])

In [170]:
import os
import wget
from tqdm import tqdm
from scipy.special import expit

if not os.path.exists('hw3_train.dat'):
    train_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw3_train.dat"
    test_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw3_test.dat"
    wget.download(train_url)
    wget.download(test_url)
train_data = np.loadtxt('hw3_train.dat')
test_data = np.loadtxt('hw3_test.dat')

X, y = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]
X = np.hstack((np.ones([X.shape[0],1]),X))
X_test = np.hstack((np.ones([X_test.shape[0],1]),X_test))
w = np.zeros([X.shape[1],1])
rate = 0.01
times = 2000
for n in tqdm(range(times)):
    # grads = np.dot(1/(np.exp(y.reshape(1,-1) * np.dot(w.T,X.T).reshape(1,-1)) + 1 ), - y.reshape((-1,1)) * X).reshape(-1,1)/y.size
    grads = np.dot(expit(-y.reshape(1,-1) * np.dot(w.T,X.T).reshape(1,-1)) , - y.reshape((-1,1)) * X).reshape(-1,1)/y.size
    # grads = expit(-y * np.dot(w.T,X.reshape(-1,1))) * (-y * X.reshape(-1,1))
    w -= rate * grads
y_predict = np.dot(X_test, w).squeeze()
print(y_predict)
print(expit(y_predict))

y_predict = np.where(expit(y_predict) > 0.5, 1.0, -1.0)

e_out = np.sum(y_predict != y_test)/y_test.size
print(e_out)



  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:00<00:00, 10350.40it/s]

[ 0.51055671 -0.04094096  0.3639267  ...  0.80367157  0.0821796
  0.27358699]
[0.62493697 0.48976619 0.58999064 ... 0.69075932 0.52053334 0.56797329]
0.22


In [ ]:
import os
import wget
from tqdm import tqdm
from scipy.special import expit

if not os.path.exists('hw3_train.dat'):
    train_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw3_train.dat"
    test_url = "https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw3_test.dat"
    wget.download(train_url)
    wget.download(test_url)
train_data = np.loadtxt('hw3_train.dat')
test_data = np.loadtxt('hw3_test.dat')

X, y = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]
X = np.hstack((np.ones([X.shape[0],1]),X))
X_test = np.hstack((np.ones([X_test.shape[0],1]),X_test))
w = np.zeros([X.shape[1],1])
rate = 0.001
times = 2000
X_original = X
y_original = y
for n in tqdm(range(times)):
    n = n % y_original.size
    X = X_original[n,:]
    y = y_original[n]
    # grads = np.dot(1/(np.exp(y.reshape(1,-1) * np.dot(w.T,X.T).reshape(1,-1)) + 1 ), - y.reshape((-1,1)) * X).reshape(-1,1)/y.size
    grads = np.dot(expit(-y.reshape(1,-1) * np.dot(w.T,X.T).reshape(1,-1)) , - y.reshape((-1,1)) * X).reshape(-1,1)/y.size
    # grads = expit(-y * np.dot(w.T,X.reshape(-1,1))) * (-y * X.reshape(-1,1))
    w -= rate * grads
y_predict = np.dot(X_test, w).squeeze()
print(y_predict)
print(expit(y_predict))

y_predict = np.where(expit(y_predict) > 0.5, 1.0, -1.0)

e_out = np.sum(y_predict != y_test)/y_test.size
print(e_out)
